In [ ]:
from astropy.io import ascii

import matplotlib.pyplot as plt

import csv
import numpy as np
import pandas as pd
import random

import glob

import warnings
warnings.filterwarnings("ignore")

import batman
from scipy.optimize import curve_fit


plt.rcParams['lines.linewidth']   = 5
plt.rcParams['axes.linewidth']    = 5
plt.rcParams['xtick.major.width'] = 5
plt.rcParams['ytick.major.width'] = 5
plt.rcParams['ytick.labelsize'] =   30
plt.rcParams['xtick.labelsize'] =   30
plt.rcParams['axes.labelsize'] =    30
plt.rcParams['legend.numpoints'] =  1
plt.rcParams['axes.labelweight']=   'semibold'
plt.rcParams['mathtext.fontset']=   'stix'
plt.rcParams['font.weight'] =       'semibold'
plt.rcParams['axes.titleweight']=   'semibold'
plt.rcParams['axes.titlesize']=     18

plt.rcParams['axes.linewidth']    = 5

plt.rcParams['xtick.minor.width'] = 4
plt.rcParams['ytick.minor.width'] = 4

plt.rcParams['xtick.minor.size'] = 8
plt.rcParams['ytick.minor.size'] = 6 

plt.rcParams['xtick.major.size'] = 8
plt.rcParams['ytick.major.size'] = 6

In [ ]:
import corner as corner
from scipy.stats import median_abs_deviation as mad

nsteps = 1000
burnin = 100
nwalkers = 10
fontsize = 28

In [ ]:
per = 8.834976 
t0 = 1821.8257 + 2457000

expected_midpoint = t0 + per*171
expected_ingress = expected_midpoint - (0.2465*24/2)/24
expected_egress = expected_midpoint + (0.2465*24/2)/24

wiggleRoom = .005

In [ ]:
def makeABatman(t, rp, t_no): # pretend everything else is measured perfectly, we really just want to check depth
    
    params = batman.TransitParams()
    params.t0 = t_no                     #time of inferior conjunction
    params.per = per                     #orbital period
    params.rp = rp                       #planet radius (in units of stellar radii)
    params.a = 11.81                     #semi-major axis (in units of stellar radii)
    params.inc = 88.7                    #orbital inclination (in degrees)
    params.ecc = 0.                      #eccentricity
    params.w = 90.                       #longitude of periastron (in degrees)
    params.u = [0.495, 0.155]            #limb darkening coefficients [u1, u2]
    
    if filt == 'gp':
        params.u = [0.92, -.087] 
        
    elif filt == 'ip':
        params.u = [0.499, 0.166]
        
    elif filt == 'zs':
        params.u = [0.4, 0.196]
        
    
    params.limb_dark = "quadratic"


    m = batman.TransitModel(params, t)    #initializes model
    flux_model = m.light_curve(params)
    
    return flux_model


def makeCorner(sampler, samples,burnin=0):
    labels = [r'rprs', r't0', r'ln(f)','a','b']    
    
    flat_samples = sampler.get_chain(discard = burnin, thin = 1, flat=True)
    
    fig = corner.corner(flat_samples, show_titles=True, labels=labels, quantiles=(0.16, 0.84), fill_contours=True, plot_datapoints = False, title_kwargs={'fontsize': 9}, title_fmt = '.3f', hist_kwargs={'linewidth': 2.5}, levels=[(1-np.exp(-0.5)), (1-np.exp(-2)), (1-np.exp(-4.5))])
    
    fig.show()
    

In [ ]:
## the likelihood and prior functions
def ln_prior(theta):
    
    rp, t0, a, b = theta
    
    if rp < 0 or rp > 1 or t0 > 0.02 or t0 < -0.02:
        return -np.inf

    sigma2 = 0.0049**2
    p1 = -0.5*((t0**2/sigma2) + np.log(2 * np.pi *sigma2))
    
    sigma2 = 0.1**2
    p2 = -0.5*(((rp-0.067)**2/sigma2) + np.log(2 * np.pi *sigma2))

    return p1+p2

def ln_likelihood(theta, t, y, yerr):

    rp, t0,a, b = theta
    model = makeABatman(t, rp, t0) 
    timeoff = t-np.min(t)
    model += a+b*timeoff
    sigma2 = yerr**2 
    
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(2 * np.pi *sigma2))


def ln_posterior(theta, t, y, yerr):
    ln_p = ln_prior(theta)
    if not np.isfinite(ln_p):
        return -np.inf
    return ln_p + ln_likelihood(theta, t, y, yerr)

def initWalkers(theta_0, ndim, nwalkers):
    
    perturbation = [0.001, 0.00001, 0.01, 0.0001] # rp, t0, a, b
    rng = np.random.default_rng()
    
    pos = []
    
    while len(pos) < nwalkers:
        p = theta_0 + perturbation*rng.standard_normal(ndim)
        
        if np.isfinite(ln_prior(p)):
            pos.append(p)
    
    return pos


from scipy.optimize import minimize
import emcee
guess = [0.068, 0, 0,0] # tess parameters -- rprs, t0,

In [ ]:
## gp fit

files = glob.glob('./LCs/*gp*.csv')
files.sort()

overallT = np.asarray([])
overallF = np.asarray([])
overallFerr = np.asarray([])

for f in files:
    data = pd.read_csv(f)
    filt = f.split('_')[1]
    label = f.split('_')[2].split('.')[0]

    time = np.asarray(data['time'])
    flux = np.asarray(data['flux'])
    fluxErr = np.asarray(data['fluxErr'])
    
    per = 8.834976 
    t0 = 1821.8257 + 2457000

    jj = 166
    while abs(t0 + per*jj - time[0]) > 1:
        jj += 1

    expected_midpoint = t0 + per*jj
    expected_ingress = expected_midpoint - (6.084/2)/24
    expected_egress = expected_midpoint + (6.084/2)/24
    
    wiggleRoom = 0.005
    
    goodPoints = (.9852 < flux) * (flux < 1.005)
    cut = np.where(goodPoints)[0]
    
    time = time[goodPoints]
    flux = flux[goodPoints]
    fluxErr = fluxErr[goodPoints]
    
    tmp1 = time - t0
    tmp2 = (tmp1/per)//1
    tmp3 = tmp1 - per*tmp2
    
    time = tmp3 - per
 
    overallT = np.concatenate((overallT, time))
    overallF = np.concatenate((overallF, flux))
    overallFerr = np.concatenate((overallFerr, fluxErr))

overallT[np.where(overallT < -per/2)] += per

l = np.argsort(overallT)

overallT = overallT[l]

overallF = overallF[l]
overallFerr = overallFerr[l]

nll = lambda *args: -ln_posterior(*args)
initial = guess
soln = minimize(nll, initial, args=(overallT, overallF, overallFerr))
ndim = np.size(initial)
theta_0 = soln.x
pos_0 = initWalkers(theta_0, ndim, nwalkers)
rprs, t0, a, b = soln.x

model = makeABatman(overallT, rprs, t0)
model += a+b*(overallT-np.min(overallT))
plt.figure(figsize=(10,8))

plt.plot(overallT*24, overallF, '.', color='darkgray', markersize = 14, alpha = .75)
plt.plot(overallT*24, model,color = 'firebrick', alpha=.8, linewidth = 6, zorder = 50)

plt.show()
# print(time)


In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, args=(overallT, overallF, overallT*0+0.0034), threads=8)

sampler.run_mcmc(pos_0, nsteps, progress=True)

samples = sampler.get_chain()

In [ ]:
# zs transits:

files = glob.glob('./LCs/*zs*.csv')
files.sort()

overallT = np.asarray([])
overallF = np.asarray([])
overallFerr = np.asarray([])

for f in files:
    data = pd.read_csv(f)
    filt = f.split('_')[1]
    label = f.split('_')[2].split('.')[0]

    time = np.asarray(data['time'])
    flux = np.asarray(data['flux'])
    fluxErr = np.asarray(data['fluxErr'])
    
    per = 8.834957
    t0 = 1821.8249 + 2457000

    jj = 166
    while abs(t0 + per*jj - time[0]) > 1:
        jj += 1

    expected_midpoint = t0 + per*jj
    expected_ingress = expected_midpoint - (6.084/2)/24
    expected_egress = expected_midpoint + (6.084/2)/24
    
    wiggleRoom = .004 + .00005*jj
    
    tmp1 = time - t0
    tmp2 = (tmp1/per)//1
    tmp3 = tmp1 - per*tmp2
    
    time = tmp3
    
    l = np.where((time*24 > 3.1) & (time<1))
    yint = np.median(flux[l])
    flux/=yint

    overallT = np.concatenate((overallT, time))
    overallF = np.concatenate((overallF, flux))
    overallFerr = np.concatenate((overallFerr, fluxErr))

overallT[np.where(overallT > per/2)] -= per    
l = np.argsort(overallT)

overallT = overallT[l]
overallF = overallF[l]
overallFerr = overallFerr[l]

time = overallT
flux = overallF
fluxErr = overallFerr

nll = lambda *args: -ln_posterior(*args)
initial = guess
soln = minimize(nll, initial, args=(overallT, overallF, overallFerr))
ndim = np.size(initial)
theta_0 = soln.x
ndim, nwalkers = ndim, 150
pos_0 = initWalkers(theta_0, ndim, nwalkers)
rprs, t0, a, b = soln.x

model = makeABatman(overallT, rprs, t0)
model += a+b*(overallT-np.min(overallT))
plt.figure(figsize=(10,8))

plt.plot(overallT*24, overallF, '.', color='darkgray', markersize = 14, alpha = .75)
plt.plot(overallT*24, model,color = 'firebrick', alpha=.8, linewidth = 6, zorder = 50)

plt.show()


In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(time, flux, fluxErr), threads=8)
sampler.run_mcmc(pos_0, nsteps, progress=True)

samples = sampler.get_chain()


In [ ]:
# ip transits 

files = glob.glob('./LCs/*ip*.csv')
files.sort()

overallT = np.asarray([])
overallF = np.asarray([])
overallFerr = np.asarray([])

for f in files:
    data = pd.read_csv(f)
    filt = f.split('_')[1]
    label = f.split('_')[2].split('.')[0]

    time = np.asarray(data['time'])
    flux = np.asarray(data['flux'])
    fluxErr = np.asarray(data['fluxErr'])
    
    per = 8.834957
    t0 = 1821.8249 + 2457000

    jj = 166
    while abs(t0 + per*jj - time[0]) > 1:
        jj += 1

    expected_midpoint = t0 + per*jj
    expected_ingress = expected_midpoint - (6.084/2)/24
    expected_egress = expected_midpoint + (6.084/2)/24
    
    wiggleRoom = .004 + .00005*jj
    
    
    tmp1 = time - t0
    tmp2 = (tmp1/per)//1
    tmp3 = tmp1 - per*tmp2
    
    time = tmp3 - per
             
    overallT = np.concatenate((overallT, time))
    overallF = np.concatenate((overallF, flux))
    overallFerr = np.concatenate((overallFerr, fluxErr))

l = np.argsort(overallT)

overallT = overallT[l]

overallF = overallF[l]
overallFerr = overallFerr[l]

time = overallT
flux = overallF
fluxErr = overallFerr


nll = lambda *args: -ln_posterior(*args)
initial = guess
soln = minimize(nll, initial, args=(overallT, overallF, overallFerr))
ndim = np.size(initial)
theta_0 = soln.x
ndim, nwalkers = ndim, 150
pos_0 = initWalkers(theta_0, ndim, nwalkers)
rprs, t0, a, b = soln.x

model = makeABatman(overallT, rprs, t0)
model += a+b*(overallT-np.min(overallT))
plt.figure(figsize=(10,8))

plt.plot(overallT*24, overallF, '.', color='darkgray', markersize = 14, alpha = .75)
plt.plot(overallT*24, model,color = 'firebrick', alpha=.8, linewidth = 6, zorder = 50)

plt.show()

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(time, flux, fluxErr), threads=8)
sampler.run_mcmc(pos_0, nsteps, progress=True)

samples = sampler.get_chain()
